# Structure Property visualizer

This is the second of two deliverables for the SiSc-Lab2020 project.

Authors = Sijie Luo and Anna Garoufali

Supervisors: Jens Bröder, Dr. Daniel Wortmann, Johannes Wasmer, Prof. Dr. Stefan Blügel.

In [ ]:
# Imports
%load_ext autoreload
%autoreload 2
%matplotlib notebook

# python imports:
from collections import Counter
import time
import numpy as np
import pandas as pd
#from pprint import pprint
# aiida imports:
from aiida import load_profile
profile = load_profile()

from aiida_jutools.sisc_lab import helpers
from bokeh.io import output_notebook

output_notebook()
helpers.print_bold(f"This notebook/dashboard will visualize the contents from the database of profile {profile.name}")
all_times = []

### Check workflows and versions

In [ ]:
t1 = time.time()
# Preprocessing: Set formula attributes for all the structure nodes
helpers.set_structure_formula()

# workflow_name = 'fleur_scf_wc' # Filter workflow
# workflowdictlst = helpers.get_structure_workflow_dict(workflow_filters={'attributes.process_label':workflow_name})
#or
workflow_name = 'FleurScfWorkChain' # No restriction. Querying by default
#workflow_filters = {'attributes.process_label' : {'==' : workflow_name}}
workflowdictlst = helpers.get_structure_workflow_dict(timing=True, check_version=True)#workflow_filters=workflow_filters, 

print("Nuumber of the workflows: ", len(workflowdictlst), '\n')
print("Workflows: ")
workflowdictlst[:2]

all_times.append(time.time()-t1)

In [ ]:
# TODO: drop down widget for selecting the workflow/dict results
# TODO: drop down widget to select property

### Check attributes

In [ ]:
# dict_project=['uuid','attributes'] # Attributes of dict nodes
# workflowdictlst = helpers.get_structure_workflow_dict(dict_project=dict_project)
# workflowdictlst[:20]
#or
# structure_project=['uuid', 'extras','attributes.kinds'] # Attributes of structure nodes
# workflowdictlst = helpers.get_structure_workflow_dict(structure_project=structure_project)
# workflowdictlst[:20]

Available dict nodes attributes for different versions of workflow


1. workflow 0.4.2
- 'workflow_version': '0.4.2',
- 'total_energy': -971.2916432694,
- 'force_largest': 0.0,
- 'distance_charge': None,
- 'total_wall_time': 176,
- 'total_energy_units': 'Htr',
- 'distance_charge_units': 'me/bohr^3',
- 'total_wall_time_units': 's'

2. Parser
- 'parser_info': 'AiiDA Fleur Parser v0.3.0',
- 'energy': -26430.191843004,
- 'bandgap': 0.0177798418,
- 'walltime': 176,
- 'energy_units': 'eV',
- 'fermi_energy': 0.2778502713,
- 'bandgap_units': 'eV',
- 'energy_hartree': -971.2916432694,
- 'walltime_units': 'seconds',
- 'fermi_energy_units': 'Htr',
- 'energy_hartree_units': 'Htr'

3. workflow 0.2.2
- 'workflow_version': '0.2.2',
- 'force': 1.241e-06,
- 'energy': -15784.56376617,
- 'energy_units': 'eV'

### Structure nodes

In [ ]:
t1 = time.time()
structure_project=['uuid', 'extras.formula']
structurenodes = helpers.generate_structure_property_pandas_source(
            workflow_name, 
            structure_project=structure_project,
            filename='structure_property.json')
#structurenodes.head()

all_times.append(time.time()-t1)

### Dict nodes

In [ ]:
t1 = time.time()
# Dict nodes with workflow_version=0.4.2
dict_project_wf042=['uuid', 'attributes.workflow_version', 'attributes.total_energy',
                    'attributes.total_energy_units', 'attributes.distance_charge',
                    'attributes.distance_charge_units', 'attributes.total_wall_time',
                    'attributes.total_wall_time_units']
dictnodes_wf042 = helpers.generate_dict_property_pandas_source(
        workflow_name, 
        dict_project=dict_project_wf042, 
        filename='dict_property_workflow042.json')
#dictnodes_wf042.head()

In [ ]:
# Dict nodes with parser of any versions
dict_project_parser=['uuid', 'attributes.parser_info', 'attributes.energy', 'attributes.energy_units', 
                     'attributes.fermi_energy', 'attributes.fermi_energy_units', 'attributes.energy_hartree', 
                     'attributes.energy_hartree_units', 'attributes.bandgap', 'attributes.bandgap_units', 
                     'attributes.walltime', 'attributes.walltime_units']
dictnodes_parser = helpers.generate_dict_property_pandas_source(
        workflow_name, 
        dict_project=dict_project_parser, 
        filename='dict_property_parser.json')
#dictnodes_parser.head()

all_times.append(time.time()-t1)

### Combine two kind of nodes

In [ ]:
t1 = time.time()
# Combined nodes with workflow_version=0.4.2
combinednodes_wf042 = helpers.generate_combination_property_pandas_source(
        workflow_name, 
        dict_project=dict_project_wf042, 
        structure_project=structure_project,
        filename='combined_property_wf042.json')
#combinednodes_wf042.head()

In [ ]:
# Combined nodes with parser of any versions
combinednodes_parser = helpers.generate_combination_property_pandas_source(
        workflow_name, 
        dict_project=dict_project_parser, 
        structure_project=structure_project,
        filename='combined_property_parser.json')
#combinednodes_parser.head()

all_times.append(time.time()-t1)

# Interactive plot

### Check data source before plotting

In [ ]:
t1 = time.time()
df = helpers.read_json_file('combined_property_parser.json')
#df

In [ ]:
filtered_df, xdata, ydata = helpers.filter_missing_value(df,'energy', 'fermi_energy')
filtered_df

In [ ]:

all_times.append(time.time()-t1)

### Interactive plot by Bokeh

In [ ]:
t1 = time.time()
# Workflow_version=0.4.2
helpers.bokeh_struc_prop_vis('combined_property_wf042.json','distance_charge', 'total_energy', "vis_wf042.html", axis_type=['log', 'linear'],nbins=40)

all_times.append(time.time()-t1)

In [ ]:
# Parser
t1 = time.time()
helpers.bokeh_struc_prop_vis('combined_property_parser.json','energy', 'bandgap',"vis_parser.html")

all_times.append(time.time()-t1)

### Interactive plot using Bokeh server application

In [ ]:
# In vscode terminal:
# bokeh serve --show --port 5001 bokehplotting.py

In [ ]:
npro  = None #sum(node_count.values())
ndata = None #sum(process_count.values())
totalnodes  = None
size = 34
header = ('# Timings of D2 in seconds\n# Database info: {} nodes; {} processes, {} data, {} MB size \n'
          '# Preprocess structures, Prepare Structure, Prepare Dict, Join Data, Loading file, Plot 1, Plot 2\n'
           ''.format(totalnodes, npro, ndata, size))
timestring = ''
for times in all_times:
    timestring = timestring + '{}  '.format(times) 
with open('all_times_D2_iffaiida.txt', 'w') as file1:
    file1.write(header)
    file1.write(timestring)